# (MBTI 별 Data 모으기: 생략) MBTI file 불러오기

In [1]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

In [2]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# tokenize and stemming

In [3]:
X_data = mbti['posts'].ravel()
y_data = mbti['type'].ravel()

print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    rm_urls = re.sub(r'http[s]?://\S+', '', sentence)
    clean_sentence = re.sub('[^A-Za-z\s]+', '', rm_urls.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [4]:
import nltk
import gc
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stop_words = set(stopwords.words('english'))
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        if word not in stop_words:
            tempData.append(ps.stem(word))
    stemData.append(tempData)
print("stemming 완료")
print(stemData[0])

#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]
del [[mbti]]
gc.collect()

stemming 시작
stemming 완료
['oi', 'went', 'break', 'month', 'ago', 'togeth', 'year', 'plan', 'life', 'around', 'relationship', 'wasnt', 'one', 'break', 'relationship', 'might', 'imagin', 'ourenfj', 'pun', 'mani', 'punswel', 'person', 'dont', 'go', 'much', 'attract', 'gener', 'see', 'chang', 'that', 'good', 'alreadi', 'may', 'ask', 'want', 'mere', 'sexualsorri', 'infp', 'im', 'realli', 'postrock', 'post', 'go', 'leav', 'sneak', 'thread', 'dthat', 'doesnt', 'sound', 'enfj', 'think', 'x', 'id', 'never', 'act', 'cold', 'toward', 'romant', 'interest', 'get', 'warm', 'side', 'think', 'like', 'someon', 'way', 'want', 'straight', 'themnot', 'realli', 'im', 'mostli', 'guitar', 'player', 'bass', 'obsess', 'im', 'person', 'ask', 'bass', 'louder', 'least', 'relev', 'like', 'tri', 'new', 'instrument', 'sometim', 'add', 'aguitar', 'mostli', 'add', 'bass', 'glockenspiel', 'harmonica', 'drum', 'ocarinaahahah', 'im', 'sorri', 'im', 'laugh', 'hard', 'realli', 'plagu', 'never', 'thought', 'possiblemostli', 

0

# vectorization (tfidf)

In [6]:
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())
X = tfidf.transform(stemData).toarray()
y = np.array(y_data)


[[0.        0.        0.        ... 0.        0.        0.       ]
 [0.0518922 0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]


# UnderSampling and SMOTE

In [7]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = RandomUnderSampler(random_state=0,
sampling_strategy={
         'ENFP': 500,
         'ENTP': 500,
         'INFJ': 500,
         'INFP': 500,
         'INTJ': 500,
         'INTP': 500,
         }).fit_sample(X, y)
X_resampled, y_resampled = SMOTE(random_state=0).fit_sample(X_resampled, y_resampled)
#format(Counter(y_resampled))
X_resampled #ndarray
Counter(y_resampled)


Counter({'ENFJ': 500,
         'ENFP': 500,
         'ENTJ': 500,
         'ENTP': 500,
         'ESFJ': 500,
         'ESFP': 500,
         'ESTJ': 500,
         'ESTP': 500,
         'INFJ': 500,
         'INFP': 500,
         'INTJ': 500,
         'INTP': 500,
         'ISFJ': 500,
         'ISFP': 500,
         'ISTJ': 500,
         'ISTP': 500})

# XGBoost

In [8]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [9]:
print("Model building Start")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)
model = XGBClassifier()
model.fit(X_train, y_train)

Model building Start


In [15]:
preds = model.predict(X_test)
print("Model building End")

Model building End


In [16]:
import joblib

joblib.dump(model, 'XGBoost_undersample.pkl')

['XGBoost_undersample.pkl']

In [17]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Evaluation Start")
# labels과 guesses
labels = preds
guesses = y_test

print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses, average=None))
print(precision_score(labels, guesses, average=None))
print(f1_score(labels, guesses, average=None))

print("Evaluation END")
# https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/

Evaluation Start
0.5159574468085106
[0.66666667 0.55555556 0.54545455 0.17391304 0.88888889 0.46153846
 0.63636364 0.77777778 0.4        0.75       0.3        0.3
 0.61538462 0.5        0.55555556 0.5       ]
[0.75       0.38461538 0.75       0.4        0.5        0.54545455
 0.46666667 0.53846154 0.25       0.69230769 0.33333333 0.3
 0.53333333 0.7        0.55555556 0.54545455]
[0.70588235 0.45454545 0.63157895 0.24242424 0.64       0.5
 0.53846154 0.63636364 0.30769231 0.72       0.31578947 0.3
 0.57142857 0.58333333 0.55555556 0.52173913]
Evaluation END
